In [1]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3

## Part 1: Scrape Coronavirus Mainland China Data from wikipedia

In [2]:
url = 'https://en.wikipedia.org/wiki/Timeline_of_the_2019%E2%80%9320_Wuhan_coronavirus_outbreak'

In [3]:
tables = pd.read_html(url)
# Since the 1st table on Wiki includes three tables, the new_confirmed table is the 4th.

### 1. New Confirmed Cases of Coronavirus in Mainland China by Provincial Divisions

In [4]:
# Since the 1st table on Wiki includes three tables, the new_confirmed table is the 4th.
# Check the infomation on in the Dataframe
df_China_new_confirmed_original = tables[3]
df_China_new_confirmed_original.info()
df_China_new_confirmed_original.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 34 columns):
Date (CST)        26 non-null object
Hubei             22 non-null object
Guangdong         18 non-null object
Beijing           17 non-null object
Shanghai          17 non-null object
Zhejiang          16 non-null object
Tianjin           16 non-null object
Chongqing         16 non-null object
Jiangxi           16 non-null object
Shandong          16 non-null object
Henan             16 non-null object
Hunan             16 non-null object
Sichuan           16 non-null object
Yunnan            15 non-null object
Shanxi            14 non-null object
Fujian            15 non-null object
Liaoning          15 non-null object
Hainan            15 non-null object
Anhui             15 non-null object
Guizhou           15 non-null object
Guangxi           15 non-null object
Ningxia           15 non-null object
Hebei             15 non-null object
Jiangsu           15 non-null object
Jilin    

,Date (CST),Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet,National,ExcludingHubei
0,2020-01-11,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41,NaN
1,2020-01-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# remove the last column of the dataframe
df_China_new_confirmed_c = df_China_new_confirmed_original.iloc[:,:-2]
# remove the last 2 rows of the dataframe
df_China_new_confirmed_r = df_China_new_confirmed_c[:-2]
# Check the tail to see if the row&column removed
df_China_new_confirmed_r.tail()

,Date (CST),Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
19,2020-01-30,1220,82,21,27,109,5,41,78,33,...,17,39,0,16,24,3,3,2,2,0
20,2020-01-31,1347,142,24,41,62,5,41,46,28,...,14,34,3,21,14,1,6,3,1,0
21,2020-02-01,1921,69,27,8,62,8,15,47,19,...,8,34,6,15,15,2,5,4,2,0
22,2020-02-02,2103,121,29,26,63,11,50,58,34,...,9,35,8,26,8,4,11,7,2,0
23,2020-02-03,2345,114,24,5,105,4,25,85,11,...,13,37,11,34,14,5,4,1,2,0


In [6]:
# # Rename the 1st column 
df_China_new_confirmed_r = df_China_new_confirmed_r.rename(columns={"Date (CST)" : "Date"})
# df_China_new_confirmed_r['Date']=df_China_new_confirmed_r['Date'].str.replace(".", "-")

df_China_new_confirmed = df_China_new_confirmed_r.copy()
# convert 'Date' column format to datetime
df_China_new_confirmed['Date'] = pd.to_datetime(df_China_new_confirmed['Date'], format= "%Y-%m-%d")
# df_China_new_confirmed.info()
# df_China_new_confirmed.head()

In [7]:
# Reset the index as the first column, this is for future use cumsum function
df_China_new_confirmed = df_China_new_confirmed.set_index(df_China_new_confirmed.columns[0])
# df_China_new_confirmed.head()

In [8]:
# fill N/A by 0
df_China_new_confirmed=df_China_new_confirmed.fillna(0)
# df_China_new_confirmed

In [33]:
df_China_new_confirmed_final = df_China_new_confirmed.astype(int) 
# df_China_new_confirmed_final.info()
# df_China_new_confirmed_final.tail()

* __The China Accumulated Confirmed Cases Table__ (Database)

In [38]:
# Sum the daliy confirmed patient
df_China_confirmed_final = df_China_new_confirmed_final.cumsum()
# Since the cumsum() function only works with integer then we change the index back as column
df_China_confirmed_final=df_China_confirmed_final.reset_index()
df_China_confirmed_final.tail()

,Date,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
19,2020-01-30,5806,393,132,128,537,32,206,240,178,...,82,168,14,59,87,17,29,20,8,1
20,2020-01-31,7153,535,156,169,599,37,247,286,206,...,96,202,17,80,101,18,35,23,9,1
21,2020-02-01,9074,604,183,177,661,45,262,333,225,...,104,236,23,95,116,20,40,27,11,1
22,2020-02-02,11177,725,212,203,724,56,312,391,259,...,113,271,31,121,124,24,51,34,13,1
23,2020-02-03,13522,839,236,208,829,60,337,476,270,...,126,308,42,155,138,29,55,35,15,1


* __The China Accumulated Confirmed Cases Transform Table__ (Database)

* __The China New Confirmed Cases Table__ (Database)

In [40]:
df_China_new_confirmed_final=df_China_new_confirmed_final.reset_index()
df_China_new_confirmed_final.tail()

,Date,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
19,2020-01-30,1220,82,21,27,109,5,41,78,33,...,17,39,0,16,24,3,3,2,2,0
20,2020-01-31,1347,142,24,41,62,5,41,46,28,...,14,34,3,21,14,1,6,3,1,0
21,2020-02-01,1921,69,27,8,62,8,15,47,19,...,8,34,6,15,15,2,5,4,2,0
22,2020-02-02,2103,121,29,26,63,11,50,58,34,...,9,35,8,26,8,4,11,7,2,0
23,2020-02-03,2345,114,24,5,105,4,25,85,11,...,13,37,11,34,14,5,4,1,2,0


### 2. New deaths from coronavirus in mainland China by provincial divisions
##### Repeat the same step as above

In [41]:
# Table of new death
df_China_new_deathes_original = tables[4]
df_China_new_deathes_original.tail()

,Date (CST),Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet,National
20,2020-01-31,45.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46[44]
21,2020-02-01,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45[49]
22,2020-02-02,56.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57[50]
23,2020-02-03,64.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64[51]
24,Net,414.0,NaN,1.0,1.0,NaN,NaN,1.0,1.0,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,425


In [42]:
# remove the last column of the dataframe
df_China_new_deathes_c = df_China_new_deathes_original.iloc[:,:-1]
# remove the last 2 rows of the dataframe
df_China_new_deathes_r = df_China_new_deathes_c[:-1]
# df_China_new_deathes_r.tail()

In [43]:
# # Rename the 1st column 
df_China_new_deathes_r = df_China_new_deathes_r.rename(columns={"Date (CST)" : "Date"})
# df_China_new_confirmed_r['Date']=df_China_new_confirmed_r['Date'].str.replace(".", "-")

df_China_new_deathes = df_China_new_deathes_r.copy()
# convert 'Date' column format to datetime
df_China_new_deathes['Date'] = pd.to_datetime(df_China_new_deathes['Date'], format= "%Y-%m-%d")

In [44]:
# Reset the index as the first column, this is for future use cumsum function
df_China_new_deathes = df_China_new_deathes.set_index(df_China_new_deathes.columns[0])

In [45]:
# fill N/A by 0
df_China_new_deathes=df_China_new_deathes.fillna(0)

In [46]:
df_China_new_deathes_final = df_China_new_deathes.astype(int) 

* __The China Accumulated Deathes Cases Table__ (Database)

In [47]:
# Sum the daliy confirmed patient
df_China_deathes_final = df_China_new_deathes_final.cumsum()
# Since the cumsum() function only works with integer then we change the index back as column
df_China_deathes_final=df_China_deathes_final.reset_index()
df_China_deathes_final.tail()

,Date,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
19,2020-01-30,204,0,1,1,0,0,0,0,0,...,1,0,0,2,0,0,0,0,0,0
20,2020-01-31,249,0,1,1,0,0,0,1,0,...,1,0,0,2,0,0,0,0,0,0
21,2020-02-01,294,0,1,1,0,0,0,1,0,...,1,0,0,2,0,0,0,0,0,0
22,2020-02-02,350,0,1,1,0,0,1,1,0,...,1,0,0,2,0,0,0,0,0,0
23,2020-02-03,414,0,1,1,0,0,1,1,0,...,1,0,0,2,0,0,0,0,0,0


* __The China New Deathes Cases Table__ (Database)

In [48]:
df_China_new_deathes_final=df_China_new_deathes_final.reset_index()
df_China_new_deathes_final.tail()

,Date,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
19,2020-01-30,42,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
20,2020-01-31,45,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
21,2020-02-01,45,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22,2020-02-02,56,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
23,2020-02-03,64,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 3. New recovered cases of coronavirus in mainland China by provincial divisions
##### Repeat the same step as above

In [49]:
# Table of new recovered
df_China_new_recovered_original = tables[5]
df_China_new_recovered_original.tail()

,Date (CST),Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet,National
20,2020-01-31,50.0,1.0,NaN,4.0,6.0,NaN,NaN,2.0,1.0,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72[44]
21,2020-02-01,49.0,2.0,4.0,1.0,8.0,NaN,2.0,1.0,2.0,...,1.0,NaN,2.0,NaN,NaN,NaN,1.0,NaN,NaN,85[50]
22,2020-02-02,80.0,2.0,3.0,NaN,13.0,1.0,4.0,8.0,1.0,...,1.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,147[50]
23,2020-02-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,Net,295.0,14.0,12.0,10.0,36.0,1.0,7.0,18.0,6.0,...,7.0,1.0,2.0,NaN,NaN,3.0,1.0,NaN,NaN,475


In [50]:
# remove the last column of the dataframe
df_China_new_recovered_c = df_China_new_recovered_original.iloc[:,:-1]
# remove the last row of the dataframe
df_China_new_recovered_r = df_China_new_recovered_c[:-1]
# df_China_new_recovered_r.tail()

In [51]:
# # Rename the 1st column 
df_China_new_recovered_r = df_China_new_recovered_r.rename(columns={"Date (CST)" : "Date"})

df_China_new_recovered = df_China_new_recovered_r.copy()
# convert 'Date' column format to datetime
df_China_new_recovered['Date'] = pd.to_datetime(df_China_new_recovered['Date'], format= "%Y-%m-%d")

In [52]:
# Reset the index as the first column, this is for future use cumsum function
df_China_new_recovered = df_China_new_recovered.set_index(df_China_new_recovered.columns[0])

In [53]:
# fill N/A by 0
df_China_new_recovered=df_China_new_recovered.fillna(0)

In [54]:
df_China_new_recovered_final = df_China_new_recovered.astype(int) 

* __The China Accumulated Recovered Cases Table__ (Database)

In [55]:
# Sum the daliy recovered patient
df_China_recovered_final = df_China_new_recovered_final.cumsum()
# Since the cumsum() function only works with integer then we change the index back as column
df_China_recovered_final=df_China_recovered_final.reset_index()
df_China_recovered_final.tail()

,Date,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
19,2020-01-30,116,9,4,6,9,0,1,7,2,...,0,2,1,0,0,0,0,0,0,0
20,2020-01-31,166,10,4,10,15,0,1,9,3,...,0,5,1,0,0,0,0,0,0,0
21,2020-02-01,215,12,8,11,23,0,3,10,5,...,0,6,1,2,0,0,0,1,0,0
22,2020-02-02,295,14,11,11,36,1,7,18,6,...,3,7,1,2,0,0,3,1,0,0
23,2020-02-03,295,14,11,11,36,1,7,18,6,...,3,7,1,2,0,0,3,1,0,0


* __The China New Recovered Cases Table__ (Database)

In [56]:
df_China_new_recovered_final=df_China_new_recovered_final.reset_index()
df_China_new_recovered_final.tail()

,Date,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
19,2020-01-30,26,3,0,1,5,0,0,4,1,...,0,1,1,0,0,0,0,0,0,0
20,2020-01-31,50,1,0,4,6,0,0,2,1,...,0,3,0,0,0,0,0,0,0,0
21,2020-02-01,49,2,4,1,8,0,2,1,2,...,0,1,0,2,0,0,0,1,0,0
22,2020-02-02,80,2,3,0,13,1,4,8,1,...,3,1,0,0,0,0,3,0,0,0
23,2020-02-03,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Part 2: Import Coronavirus Mainland China Data Tables to SQLite

* __Table 1: The China Accumulated Confirmed Cases Table__ (Database)

### __Note : PK will add to sqlite by using "DB Broswer for SQLite"__

In [57]:
df_China_confirmed_final.columns

Index(['Date', 'Hubei', 'Guangdong', 'Beijing', 'Shanghai', 'Zhejiang',
       'Tianjin', 'Chongqing', 'Jiangxi', 'Shandong', 'Henan', 'Hunan',
       'Sichuan', 'Yunnan', 'Shanxi', 'Fujian', 'Liaoning', 'Hainan', 'Anhui',
       'Guizhou', 'Guangxi', 'Ningxia', 'Hebei', 'Jiangsu', 'Jilin',
       'Heilongjiang', 'Shaanxi', 'Xinjiang', 'Gansu', 'InnerMongolia',
       'Qinghai', 'Tibet'],
      dtype='object')

In [58]:
df_China_confirmed_final.head()

,Date,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
0,2020-01-11,41,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2020-01-12,41,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2020-01-13,41,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2020-01-14,41,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2020-01-15,41,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# conn = sqlite3.connect('wuhan_pneumonia.sqlite')
# c = conn.cursor()
# c.execute("DROP TABLE IF EXISTS China_confirmed;")

In [ ]:
# c.execute('''
#           CREATE TABLE China_confirmed
#           ([Date] TEXT PRIMARY KEY, 
#            [Hubei] INTEGER NOT NULL,
#            [Guangdong] INTEGER NOT NULL,
#            [Beijing] INTEGER NOT NULL,
#            [Shanghai] INTEGER NOT NULL, 
#            [Zhejiang] INTEGER NOT NULL,
#            [Tianjin] INTEGER NOT NULL, 
#            [Chongqing] INTEGER NOT NULL, 
#            [Jiangxi] INTEGER NOT NULL, 
#            [Shandong] INTEGER NOT NULL, 
#            [Henan] INTEGER NOT NULL, 
#            [Hunan] INTEGER NOT NULL,
#            [Sichuan] INTEGER NOT NULL, 
#            [Yunnan] INTEGER NOT NULL, 
#            [Shanxi] INTEGER NOT NULL, 
#            [Fujian] INTEGER NOT NULL, 
#            [Liaoning] INTEGER NOT NULL, 
#            [Hainan] INTEGER NOT NULL, 
#            [Anhui] INTEGER NOT NULL,
#            [Guizhou] INTEGER NOT NULL, 
#            [Guangxi] INTEGER NOT NULL, 
#            [Ningxia] INTEGER NOT NULL, 
#            [Hebei] INTEGER NOT NULL, 
#            [Jiangsu] INTEGER NOT NULL, 
#            [Jilin] INTEGER NOT NULL,
#            [Heilongjiang] INTEGER NOT NULL, 
#            [Shaanxi] INTEGER NOT NULL, 
#            [Xinjiang] INTEGER NOT NULL, 
#            [Gansu] INTEGER NOT NULL, 
#            [InnerMongolia] INTEGER NOT NULL,
#            [Qinghai] INTEGER NOT NULL, 
#            [Tibet] INTEGER NOT NULL
#            )
#           ''')
# conn.commit()
# conn.close()

In [59]:
engine = create_engine('sqlite:///wuhan_pneumonia.sqlite')
df_China_confirmed_final.to_sql(name='China_confirmed', con=engine, index=False , if_exists='replace')

In [60]:
df_confirmed_China = pd.read_sql_query('SELECT * FROM China_confirmed',engine)
df_confirmed_China.tail()

,Date,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
19,2020-01-30 00:00:00.000000,5806,393,132,128,537,32,206,240,178,...,82,168,14,59,87,17,29,20,8,1
20,2020-01-31 00:00:00.000000,7153,535,156,169,599,37,247,286,206,...,96,202,17,80,101,18,35,23,9,1
21,2020-02-01 00:00:00.000000,9074,604,183,177,661,45,262,333,225,...,104,236,23,95,116,20,40,27,11,1
22,2020-02-02 00:00:00.000000,11177,725,212,203,724,56,312,391,259,...,113,271,31,121,124,24,51,34,13,1
23,2020-02-03 00:00:00.000000,13522,839,236,208,829,60,337,476,270,...,126,308,42,155,138,29,55,35,15,1


* __Table 2: The China New Confirmed Cases Table__ (Database)

In [61]:
df_China_new_confirmed_final.to_sql('China_new_confirmed', engine, index=False,if_exists='replace')

In [62]:
df_new_confirmed_China = pd.read_sql_query('SELECT * FROM China_new_confirmed',engine)
df_new_confirmed_China.tail()

,Date,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
19,2020-01-30 00:00:00.000000,1220,82,21,27,109,5,41,78,33,...,17,39,0,16,24,3,3,2,2,0
20,2020-01-31 00:00:00.000000,1347,142,24,41,62,5,41,46,28,...,14,34,3,21,14,1,6,3,1,0
21,2020-02-01 00:00:00.000000,1921,69,27,8,62,8,15,47,19,...,8,34,6,15,15,2,5,4,2,0
22,2020-02-02 00:00:00.000000,2103,121,29,26,63,11,50,58,34,...,9,35,8,26,8,4,11,7,2,0
23,2020-02-03 00:00:00.000000,2345,114,24,5,105,4,25,85,11,...,13,37,11,34,14,5,4,1,2,0


* __Table 3: The China Accumulated Deathes Cases Table__ (Database)

In [63]:
df_China_deathes_final.to_sql('China_deathes', engine, index=False,if_exists='replace')

In [64]:
df_deathes_China = pd.read_sql_query('SELECT * FROM China_deathes',engine)
df_deathes_China.tail()

,Date,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
19,2020-01-30 00:00:00.000000,204,0,1,1,0,0,0,0,0,...,1,0,0,2,0,0,0,0,0,0
20,2020-01-31 00:00:00.000000,249,0,1,1,0,0,0,1,0,...,1,0,0,2,0,0,0,0,0,0
21,2020-02-01 00:00:00.000000,294,0,1,1,0,0,0,1,0,...,1,0,0,2,0,0,0,0,0,0
22,2020-02-02 00:00:00.000000,350,0,1,1,0,0,1,1,0,...,1,0,0,2,0,0,0,0,0,0
23,2020-02-03 00:00:00.000000,414,0,1,1,0,0,1,1,0,...,1,0,0,2,0,0,0,0,0,0


* __Table 4: The China New Deathes Cases Table__ (Database)

In [65]:
df_China_new_deathes_final.to_sql('China_new_deathes', engine, index=False,if_exists='replace')

In [66]:
df_new_deathes_China = pd.read_sql_query('SELECT * FROM China_new_deathes',engine)
df_new_deathes_China.tail()

,Date,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
19,2020-01-30 00:00:00.000000,42,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
20,2020-01-31 00:00:00.000000,45,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
21,2020-02-01 00:00:00.000000,45,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22,2020-02-02 00:00:00.000000,56,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
23,2020-02-03 00:00:00.000000,64,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


* __Table 5: The China Accumulated Recovered Cases Table__ (Database)

In [67]:
df_China_recovered_final.to_sql('China_recovered', engine, index=False,if_exists='replace')

In [68]:
df_recovered_China = pd.read_sql_query('SELECT * FROM China_recovered',engine)
df_recovered_China.tail()

,Date,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
19,2020-01-30 00:00:00.000000,116,9,4,6,9,0,1,7,2,...,0,2,1,0,0,0,0,0,0,0
20,2020-01-31 00:00:00.000000,166,10,4,10,15,0,1,9,3,...,0,5,1,0,0,0,0,0,0,0
21,2020-02-01 00:00:00.000000,215,12,8,11,23,0,3,10,5,...,0,6,1,2,0,0,0,1,0,0
22,2020-02-02 00:00:00.000000,295,14,11,11,36,1,7,18,6,...,3,7,1,2,0,0,3,1,0,0
23,2020-02-03 00:00:00.000000,295,14,11,11,36,1,7,18,6,...,3,7,1,2,0,0,3,1,0,0


* __Table 6: The China New Recovered Cases Table__ (Database)

In [69]:
df_China_new_recovered_final.to_sql('China_new_recovered', engine, index=False,if_exists='replace')

In [70]:
df_new_recovered_China = pd.read_sql_query('SELECT * FROM China_new_recovered',engine)
df_new_recovered_China.tail()

,Date,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
19,2020-01-30 00:00:00.000000,26,3,0,1,5,0,0,4,1,...,0,1,1,0,0,0,0,0,0,0
20,2020-01-31 00:00:00.000000,50,1,0,4,6,0,0,2,1,...,0,3,0,0,0,0,0,0,0,0
21,2020-02-01 00:00:00.000000,49,2,4,1,8,0,2,1,2,...,0,1,0,2,0,0,0,1,0,0
22,2020-02-02 00:00:00.000000,80,2,3,0,13,1,4,8,1,...,3,1,0,0,0,0,3,0,0,0
23,2020-02-03 00:00:00.000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


* __Table 7: The China confirmed Cases Transformed Table__ (Database)

In [73]:
df_confirmed_China['Date'] = df_confirmed_China['Date'].str[0:10]
df_confirmed_China = df_confirmed_China.set_index("Date")
df_confirmed_China.tail()

,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,Henan,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-30,5806,393,132,128,537,32,206,240,178,352,...,82,168,14,59,87,17,29,20,8,1
2020-01-31,7153,535,156,169,599,37,247,286,206,422,...,96,202,17,80,101,18,35,23,9,1
2020-02-01,9074,604,183,177,661,45,262,333,225,493,...,104,236,23,95,116,20,40,27,11,1
2020-02-02,11177,725,212,203,724,56,312,391,259,566,...,113,271,31,121,124,24,51,34,13,1
2020-02-03,13522,839,236,208,829,60,337,476,270,675,...,126,308,42,155,138,29,55,35,15,1


In [74]:
df_confirmed_China_stacked = df_confirmed_China.stack()
df_confirmed_China_stacked_new = pd.DataFrame([df_confirmed_China_stacked]).T
df_confirmed_China_stacked_new.tail()

0
Date                        
2020-02-03 Xinjiang       29
           Gansu          55
           InnerMongolia  35
           Qinghai        15
           Tibet           1

In [75]:
df_confirmed_China_stacked_final = df_confirmed_China_stacked_new.reset_index(level=[0,1])
df_confirmed_China_stacked_final.columns = ['Date', 'Province', 'Confirmed']
df_confirmed_China_final = df_confirmed_China_stacked_final.set_index('Province')
df_confirmed_China_final.tail()

,Date,Confirmed
Province,,
Xinjiang,2020-02-03,29
Gansu,2020-02-03,55
InnerMongolia,2020-02-03,35
Qinghai,2020-02-03,15
Tibet,2020-02-03,1


In [76]:
df_new_confirmed_China = pd.read_sql_query('SELECT * FROM China_new_confirmed',engine)

In [77]:
df_new_confirmed_China['Date'] = df_new_confirmed_China['Date'].str[0:10]
df_new_confirmed_China = df_new_confirmed_China.set_index("Date")
df_new_confirmed_China.tail()

,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,Henan,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-30,1220,82,21,27,109,5,41,78,33,74,...,17,39,0,16,24,3,3,2,2,0
2020-01-31,1347,142,24,41,62,5,41,46,28,70,...,14,34,3,21,14,1,6,3,1,0
2020-02-01,1921,69,27,8,62,8,15,47,19,71,...,8,34,6,15,15,2,5,4,2,0
2020-02-02,2103,121,29,26,63,11,50,58,34,73,...,9,35,8,26,8,4,11,7,2,0
2020-02-03,2345,114,24,5,105,4,25,85,11,109,...,13,37,11,34,14,5,4,1,2,0


In [78]:
df_new_confirmed_China_stacked = df_new_confirmed_China.stack()
df_new_confirmed_China_stacked_new = pd.DataFrame([df_new_confirmed_China_stacked]).T
df_new_confirmed_China_stacked_new.tail()

0
Date                       
2020-02-03 Xinjiang       5
           Gansu          4
           InnerMongolia  1
           Qinghai        2
           Tibet          0

In [79]:
df_new_confirmed_China_stacked_final = df_new_confirmed_China_stacked_new.reset_index(level=[0,1])
df_new_confirmed_China_stacked_final.columns = ['Date', 'Province', 'New Confirmed']
df_new_confirmed_China_final = df_new_confirmed_China_stacked_final.set_index('Province')
df_new_confirmed_China_final.tail()

,Date,New Confirmed
Province,,
Xinjiang,2020-02-03,5
Gansu,2020-02-03,4
InnerMongolia,2020-02-03,1
Qinghai,2020-02-03,2
Tibet,2020-02-03,0


In [80]:
df_deathes_China = pd.read_sql_query('SELECT * FROM China_deathes',engine)

In [81]:
df_deathes_China['Date'] = df_deathes_China['Date'].str[0:10]
df_deathes_China = df_deathes_China.set_index("Date")
df_deathes_China.tail()

,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,Henan,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-30,204,0,1,1,0,0,0,0,0,2,...,1,0,0,2,0,0,0,0,0,0
2020-01-31,249,0,1,1,0,0,0,1,0,2,...,1,0,0,2,0,0,0,0,0,0
2020-02-01,294,0,1,1,0,0,0,1,0,2,...,1,0,0,2,0,0,0,0,0,0
2020-02-02,350,0,1,1,0,0,1,1,0,2,...,1,0,0,2,0,0,0,0,0,0
2020-02-03,414,0,1,1,0,0,1,1,0,2,...,1,0,0,2,0,0,0,0,0,0


In [82]:
df_deathes_China_stacked = df_deathes_China.stack()
df_deathes_China_stacked_new = pd.DataFrame([df_deathes_China_stacked]).T
df_deathes_China_stacked_new.tail()

0
Date                       
2020-02-03 Xinjiang       0
           Gansu          0
           InnerMongolia  0
           Qinghai        0
           Tibet          0

In [83]:
df_deathes_China_stacked_final =df_deathes_China_stacked_new.reset_index(level=[0,1])
df_deathes_China_stacked_final.columns = ['Date', 'Province', 'Death']
df_deathes_China_final = df_deathes_China_stacked_final.set_index('Province')
df_deathes_China_final.tail()

,Date,Death
Province,,
Xinjiang,2020-02-03,0
Gansu,2020-02-03,0
InnerMongolia,2020-02-03,0
Qinghai,2020-02-03,0
Tibet,2020-02-03,0


In [84]:
df_new_deathes_China = pd.read_sql_query('SELECT * FROM China_new_deathes',engine)

In [85]:
df_new_deathes_China['Date'] = df_new_deathes_China['Date'].str[0:10]
df_new_deathes_China = df_new_deathes_China.set_index("Date")
df_new_deathes_China.tail()

,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,Henan,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-30,42,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2020-01-31,45,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-02-01,45,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-02-02,56,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-02-03,64,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
df_new_deathes_China_stacked = df_new_deathes_China.stack()
df_new_deathes_China_stacked_new = pd.DataFrame([df_new_deathes_China_stacked]).T
df_new_deathes_China_stacked_new.tail()

0
Date                       
2020-02-03 Xinjiang       0
           Gansu          0
           InnerMongolia  0
           Qinghai        0
           Tibet          0

In [87]:
df_new_deathes_China_stacked_final =df_new_deathes_China_stacked_new.reset_index(level=[0,1])
df_new_deathes_China_stacked_final.columns = ['Date', 'Province', 'New Death']
df_new_deathes_China_final = df_new_deathes_China_stacked_final.set_index('Province')
df_new_deathes_China_final.tail()

,Date,New Death
Province,,
Xinjiang,2020-02-03,0
Gansu,2020-02-03,0
InnerMongolia,2020-02-03,0
Qinghai,2020-02-03,0
Tibet,2020-02-03,0


In [88]:
df_recovered_China = pd.read_sql_query('SELECT * FROM China_recovered',engine)

In [89]:
df_recovered_China['Date'] = df_recovered_China['Date'].str[0:10]
df_recovered_China = df_recovered_China.set_index("Date")
df_recovered_China.tail()

,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,Henan,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-30,116,9,4,6,9,0,1,7,2,2,...,0,2,1,0,0,0,0,0,0,0
2020-01-31,166,10,4,10,15,0,1,9,3,2,...,0,5,1,0,0,0,0,0,0,0
2020-02-01,215,12,8,11,23,0,3,10,5,3,...,0,6,1,2,0,0,0,1,0,0
2020-02-02,295,14,11,11,36,1,7,18,6,13,...,3,7,1,2,0,0,3,1,0,0
2020-02-03,295,14,11,11,36,1,7,18,6,13,...,3,7,1,2,0,0,3,1,0,0


In [90]:
df_recovered_China_stacked = df_recovered_China.stack()
df_recovered_China_stacked_new = pd.DataFrame([df_recovered_China_stacked]).T
df_recovered_China_stacked_new.tail()

0
Date                       
2020-02-03 Xinjiang       0
           Gansu          3
           InnerMongolia  1
           Qinghai        0
           Tibet          0

In [91]:
df_recovered_China_stacked_final =df_recovered_China_stacked_new.reset_index(level=[0,1])
df_recovered_China_stacked_final.columns = ['Date', 'Province', 'Recovered']
df_recovered_China_final = df_recovered_China_stacked_final.set_index('Province')
df_recovered_China_final.head()

,Date,Recovered
Province,,
Hubei,2020-01-11,2
Guangdong,2020-01-11,0
Beijing,2020-01-11,0
Shanghai,2020-01-11,0
Zhejiang,2020-01-11,0


In [92]:
df_new_recovered_China = pd.read_sql_query('SELECT * FROM China_new_recovered',engine)

In [93]:
df_new_recovered_China['Date'] = df_new_recovered_China['Date'].str[0:10]
df_new_recovered_China = df_new_recovered_China.set_index("Date")

In [94]:
df_new_recovered_China_stacked = df_new_recovered_China.stack()
df_new_recovered_China_stacked_new = pd.DataFrame([df_new_recovered_China_stacked]).T

In [95]:
df_new_recovered_China_stacked_final =df_new_recovered_China_stacked_new.reset_index(level=[0,1])
df_new_recovered_China_stacked_final.columns = ['Date', 'Province', 'New Recovered']
df_new_recovered_China_final = df_new_recovered_China_stacked_final.set_index('Province')
df_new_recovered_China_final.head()

,Date,New Recovered
Province,,
Hubei,2020-01-11,2
Guangdong,2020-01-11,0
Beijing,2020-01-11,0
Shanghai,2020-01-11,0
Zhejiang,2020-01-11,0


In [96]:
df_merge1 = pd.merge(df_confirmed_China_final, df_new_confirmed_China_final, on=['Province', 'Date'])
df_merge2 = pd.merge(df_deathes_China_final, df_new_deathes_China_final, on=['Province', 'Date'])
df_merge3 = pd.merge(df_recovered_China_final, df_new_recovered_China_final, on=['Province', 'Date'])
df_merge4 = pd.merge(df_merge1, df_merge2, on=['Province', 'Date'])
df_merge5 = pd.merge(df_merge4, df_merge3, on=['Province', 'Date'])

In [97]:
df_merge5 = df_merge5.reset_index()
df_merge5['Country'] ='China'
df_china_current_table = df_merge5[['Date','Province','Country','Confirmed','New Confirmed','Death','New Death','Recovered','New Recovered']]
df_china_current_table

,Date,Province,Country,Confirmed,New Confirmed,Death,New Death,Recovered,New Recovered
0,2020-01-11,Hubei,China,41,41,1,1,2,2
1,2020-01-11,Guangdong,China,0,0,0,0,0,0
2,2020-01-11,Beijing,China,0,0,0,0,0,0
3,2020-01-11,Shanghai,China,0,0,0,0,0,0
4,2020-01-11,Zhejiang,China,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
739,2020-02-03,Xinjiang,China,29,5,0,0,0,0
740,2020-02-03,Gansu,China,55,4,0,0,3,0
741,2020-02-03,InnerMongolia,China,35,1,0,0,1,0
742,2020-02-03,Qinghai,China,15,2,0,0,0,0


In [104]:
df_china_current_table.to_sql('China_current_final', engine, index=True,if_exists='replace')

In [105]:
df_China_transformed = pd.read_sql_query('SELECT * FROM China_current_final',engine)
df_China_transformed.tail()

,index,Date,Province,Country,Confirmed,New Confirmed,Death,New Death,Recovered,New Recovered
739,739,2020-02-03,Xinjiang,China,29,5,0,0,0,0
740,740,2020-02-03,Gansu,China,55,4,0,0,3,0
741,741,2020-02-03,InnerMongolia,China,35,1,0,0,1,0
742,742,2020-02-03,Qinghai,China,15,2,0,0,0,0
743,743,2020-02-03,Tibet,China,1,0,0,0,0,0
